In [1]:
import torch

In [2]:
torch.cuda.is_available()

True

In [3]:
if torch.cuda.is_available():
    print("CUDA is available. You can use GPU for computation.")
else:
    print("CUDA is not available. Only CPU will be used for computation.")

CUDA is available. You can use GPU for computation.


In [4]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
from torch import cuda
assert cuda.is_available()
assert cuda.device_count() > 0
print(cuda.get_device_name(cuda.current_device()))

NVIDIA GeForce RTX 2060 SUPER


In [13]:
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import cv2
import os
from glob import glob

# 0. 경로 설정
image_folder = "C:/Users/rund6/Desktop/1030/images_removebg"  # 이미지가 저장된 폴더 경로
output_path = "C:/Users/rund6/Desktop/1030/custom_image_features.csv"

# 1. 이미지 전처리 및 Dataset 정의
class CustomImageDataset(Dataset):
    def __init__(self, image_folder, transform=None):
        self.image_paths = glob(os.path.join(image_folder, "*.png"))  # PNG 파일 사용
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = cv2.imread(img_path, cv2.IMREAD_COLOR)  # 컬러 이미지로 로드
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # RGB로 변환 (PyTorch는 RGB 포맷을 사용)
        
         # 디버깅 코드: 이미지 경로와 이미지 크기를 출력
        print(f"Loading image from path: {img_path}")
        print(f"Image shape before transform: {image.shape}")
        
        if self.transform:
            image = self.transform(image)
        
        label = os.path.basename(img_path)  # 파일명을 레이블로 사용
        return image, label

# 전처리 설정 (28x28 크기 및 정규화)
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((28, 28)),
    transforms.ToTensor()
])

dataset = CustomImageDataset(image_folder=image_folder, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

# 2. CNN 모델 정의
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)  # RGB이므로 입력 채널 수는 3
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.pool(x)
        x = torch.relu(self.conv2(x))
        x = self.pool(x)
        x = self.flatten(x)
        return x

# 모델 생성 및 특징 추출
model = CNNModel()
model.eval()

# 3. 이미지 데이터로부터 특징 추출 및 CSV 저장
all_features = []
all_labels = []

with torch.no_grad():
    for images, labels in dataloader:
        features = model(images)  # 특징 추출
        all_features.extend(features.numpy())  # 특징을 리스트에 추가
        all_labels.extend(labels)  # 레이블 추가

# 4. DataFrame으로 변환 및 CSV 파일로 저장
features_df = pd.DataFrame(all_features)
features_df['label'] = all_labels  # 각 이미지의 파일명을 레이블로 사용
features_df.to_csv(output_path, index=False)

print(f"컬러 이미지에서 추출된 특징이 {output_path} 파일로 저장되었습니다.")

Loading image from path: C:/Users/rund6/Desktop/1030/images_removebg\output_202301020809249029030865.png
Image shape before transform: (1333, 1000, 3)
Loading image from path: C:/Users/rund6/Desktop/1030/images_removebg\output_202301020904024489037100.png
Image shape before transform: (1333, 1000, 3)
Loading image from path: C:/Users/rund6/Desktop/1030/images_removebg\output_202301020947419689041463.png
Image shape before transform: (1333, 1000, 3)
Loading image from path: C:/Users/rund6/Desktop/1030/images_removebg\output_202301030642200889013015.png
Image shape before transform: (1333, 1000, 3)
Loading image from path: C:/Users/rund6/Desktop/1030/images_removebg\output_202301031117366569052863.png
Image shape before transform: (1333, 1000, 3)
Loading image from path: C:/Users/rund6/Desktop/1030/images_removebg\output_202301031129051719027417.png
Image shape before transform: (1333, 1000, 3)
Loading image from path: C:/Users/rund6/Desktop/1030/images_removebg\output_202301040956122179

In [8]:
print(all_features)


[]


In [11]:
import os
print("Image folder exists:", os.path.exists(image_folder))

Image folder exists: True
